In [3]:
# import dependencies 
import pandas as pd 
import matplotlib.pyplot as plt
import random
import datetime
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from statistics import mean
from nltk import word_tokenize, pos_tag
import psycopg2
import time
from sqlalchemy import create_engine
from collections import Counter

### Important: Manually Enter Database Names and Passwords Below

In [4]:
# creating connection string components for SQLAlchemy
host = '127.0.0.1'
port = 5432
user = 'postgres'
passw = ""
database = ""

In [5]:
# compile connection string components and create engine
db_string = f"postgresql+psycopg2://{user}:{passw}@{host}:{port}/{database}"
engine = create_engine(db_string, pool_recycle=3600)
# Connect to server
dbConnection = engine.connect()

In [10]:
def sentiment_etl():
    # ticker list manually curated via top trending tickers from apewisdom.io in May 2022
    ticker_ls = ['GME','TSLA','AAPL','TWTR','ATER','AMC','AMD','MSFT','HD','FB','AMZN','NU',
                'DTE','GOOG','NFLX','SOFI','TA','BBIG','NVDA','AI','ZIM','VTNR','PLTR','RIVN',
                'LUNA','TTWO','OXY','BABA','WMT','DWAC','CC','COO','LFG','RBLX','DIS','SE',
                'SNDL','PARA','SP','XOM','ES','JD','PTON','WBD','TGT','OG','DM','DOW','TLRY',
                'NIO','PANW','CVNA','JPM','IP','TD','GS','CLOV','PYPL','GM','IQ','MCD','MULN',
                'DE','VZ','WW','EA','BB','CRSR','TG','WEBR','ABNB','OI','CHGG','INTC','HP',
                'SQ','ROKU','PT','VXX','BYND','JBLU','GBTC','MS','DKNG','FCF','GE','BJ','LCID',
                'CS','KO','WTI','RIG','RC','BA','MMAT','RKLB','UNH','RE','CSCO','VC','ZM','TIL',
                'IBKR','CRM','IBM','ET','CP','MSTR','DAC','DTC','NYT','TH','BP','LMT','SM','ITT',
                'BBBY','TTD','PBR','SPCE','UPST','AFRM','NG']
    
    # positive string based on manual labelling / analysis of pulled data
    pos_strings = ['calls', 'buy', 'bought', 'buying', 'all in', 'to the moon', 'long',
                  'up', 'green', 'rocket', 'pop', 'bullish', 'bull', 'rally', 'beat', 'beats',
                  'crush', 'upswing', 'tendies', 'profit', 'undervalued', 'rallies']

    neg_strings = ['puts', 'pain', 'sell', 'sold', 'short', 'down', 'tank', 'red', 'dip',
                  'bearish', 'bear', 'miss', 'selloff', 'underperform', 'drop', 'downswing', 
                  'overvalued', 'fucked', 'screwed']
    
    # transform DF from one to many per ticker present in row text 
    def transform_to_many(df):
        new_df = pd.DataFrame(columns = ["comment_id", "ticker", "date", "username", "subreddit", "body"])
        for i in range(len(df.index)):
            row = df.loc[i]
            text = row.body

            for ticker in ticker_ls:
                search_string = fr"\b\$?({ticker.lower()})\b"
                mo = re.search(search_string, text.lower())
                if mo:
                    new_row = pd.DataFrame([[row.comment_id, ticker, row.date, row.username, row.subreddit, row.body]],
                                          columns = ["comment_id", "ticker", "date", "username", "subreddit", "body"])
                    new_df = pd.concat([new_df, new_row], sort=False)
        return new_df
    
    
    def add_sentiment_data(df):
        # initialize nltk sentiment analyzer
        sia = SentimentIntensityAnalyzer()
        # overall sentiment compound scores
        overall_sent_comp = [sia.polarity_scores(txt)['compound'] for txt in df['body']]
        # overall sentiment positive scores
        overall_sent_pos = [sia.polarity_scores(txt)['pos'] for txt in df['body']]
        # overall sentiment negative scores
        overall_sent_neg = [sia.polarity_scores(txt)['neg'] for txt in df['body']]

        # polarity scores for TOKENIZED sentences
        sent_tokens = [nltk.sent_tokenize(txt) for txt in df['body']]
        pol_scores = [[sia.polarity_scores(sent) for sent in txt] for txt in sent_tokens]

        # mean scores for tokenized sentences
        mean_t_comp_scores = [mean([token["compound"] for token in item]) for item in pol_scores]
        mean_t_pos_scores = [mean([token["pos"] for token in item]) for item in pol_scores]
        mean_t_neg_scores = [mean([token["neg"] for token in item]) for item in pol_scores]  

        # filter sentence tokens based on corresponding ticker
        tickers = df['ticker']
        tgt_tokens = [[sent for sent in txt if ticker.lower() in sent.lower()] for txt, ticker in zip(sent_tokens, tickers)]

        # generate polarity scores based on targeted sentence tokens
        target_pol_scores = [[sia.polarity_scores(sent) for sent in txt] for txt in tgt_tokens]
        
        # compute mean scores per token sentences
        mean_tgt_comp_scores = [mean([token["compound"] for token in item]) for item in target_pol_scores]
        mean_tgt_pos_scores = [mean([token["pos"] for token in item]) for item in target_pol_scores]
        mean_tgt_neg_scores = [mean([token["neg"] for token in item]) for item in target_pol_scores]


        # estimate average verb tense of text 
        def determine_tense(sentence):
            text = word_tokenize(sentence)
            tagged = pos_tag(text)

            tense = {}
            tense["future"] = len([word for word in tagged if word[1] == "MD"])
            tense["present"] = len([word for word in tagged if word[1] in ["VBP", "VBZ","VBG"]])
            tense["past"] = len([word for word in tagged if word[1] in ["VBD", "VBN"]]) 
            return(max(tense, key=tense.get))

        # get maximum value of verb tense counter to estimate average verb tense
        verb_tenses = [max(Counter([determine_tense(sent) for sent in txt])) for txt in tgt_tokens]

        # determines sentiment value based on custom strings
        def custom_sent(sentence):
            sent_vals = {"pos":0, "neg":0}
            for word in pos_strings:
                if word in sentence.lower():
                    sent_vals['pos'] += 1
            for word in neg_strings:
                if word in sentence.lower():
                    sent_vals['neg'] += 1

            total = sent_vals['pos'] + sent_vals['neg']

            if total > 0:
                return (sent_vals['pos'] + -1 * (sent_vals['neg'])) / total
            else:
                return 0

        mean_custom_scores = [mean([custom_sent(sent) for sent in txt]) for txt in tgt_tokens]

        df['overall_sent_comp'] = overall_sent_comp
        df['overall_sent_pos'] = overall_sent_pos
        df['overall_sent_neg'] = overall_sent_neg
        df['mean_t_comp_score'] = mean_t_comp_scores
        df['mean_t_pos_score'] = mean_t_pos_scores
        df['mean_t_neg_score'] = mean_t_neg_scores
        df['mean_tgt_comp_score'] = mean_tgt_comp_scores
        df['mean_tgt_pos_score'] = mean_tgt_pos_scores
        df['mean_tgt_neg_score'] = mean_tgt_neg_scores

        df['verb_tense'] = verb_tenses
        df['mean_custom_score'] = mean_custom_scores
        return df
    
    rows_imported = 0
    start_time = time.time()
    for data in pd.read_sql("select * from \"comments\"", dbConnection, chunksize = 10000):

        print(f'importing rows {rows_imported} to {rows_imported + len(data)}...\n', end='')

        data = transform_to_many(data)
        data = add_sentiment_data(data)
        
        data.to_sql(name='sentiment', con=engine, if_exists='append', index=False)
        rows_imported += len(data)
    print(f'Done. {time.time() - start_time} total seconds elapsed')

In [ ]:
sentiment_etl()

importing rows 0 to 10000...
importing rows 15215 to 25215...
importing rows 28131 to 38131...
importing rows 39945 to 49945...
importing rows 52915 to 62915...
importing rows 65042 to 75042...
importing rows 76941 to 86941...
importing rows 89604 to 99604...
importing rows 102145 to 112145...
importing rows 115353 to 125353...
importing rows 128377 to 138377...
importing rows 142087 to 152087...
importing rows 155064 to 165064...
importing rows 166417 to 176417...
importing rows 178521 to 188521...
importing rows 190762 to 200762...
importing rows 203294 to 213294...
importing rows 216323 to 226323...
importing rows 227877 to 237877...
importing rows 240246 to 250246...
importing rows 253542 to 263542...
importing rows 281523 to 291523...
importing rows 295426 to 305426...
importing rows 308838 to 318838...
importing rows 320200 to 330200...
importing rows 332077 to 342077...
importing rows 344096 to 354096...
importing rows 356552 to 366552...
importing rows 369148 to 379148...
impor